In [1]:
import numpy as np
import librosa
import pandas as pd
import IPython.display as ipd
import pickle

In [2]:
def findEmotion(fileName):
    
    emotionMap = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fearful', 7:'disgust', 8:'surprised'}
    finalModel = pickle.load(open("finalModel.pickle.dat", "rb"))
    def featureExtraction(path):
        X, sample_rate = librosa.load(path)
        stft = np.abs(librosa.stft(X))
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        return mfccs, chroma, mel, contrast, tonnetz
    cols = ['emotion', 'intensity', 'gender', 'chr_mn', 'chr_std', 'cont_mn', 'cont_std', 'ton_mn', 'ton_std', 'mel02_mn', 'mel02_std', 'mel24_mn', 'mel24_std', 'mel46_mn', 'mel46_std', 'mel68_mn', 'mel68_std']
    initDict = dict()
    for i in cols:
        initDict[i] = list()
    df = pd.DataFrame(initDict)
    mfccs, chroma, mel, contrast, tonnetz = featureExtraction(fileName)
    details = dict()
    details['chr_mn'] = np.mean(chroma)
    details['chr_std'] = np.std(chroma)
    details['cont_mn'] = np.mean(contrast)
    details['cont_std'] = np.std(contrast)
    details['ton_mn'] = np.mean(tonnetz)
    details['ton_std'] = np.std(tonnetz)
    details['mel02_mn'] = np.mean(mel[0:20])
    details['mel02_std'] = np.std(mel[0:20])
    details['mel24_mn'] = np.mean(mel[20:40])
    details['mel24_std'] = np.std(mel[20:40])
    details['mel46_mn'] = np.mean(mel[40:60])
    details['mel46_std'] = np.std(mel[40:60])
    details['mel68_mn'] = np.mean(mel[60:80])
    details['mel68_std'] = np.std(mel[60:80])
    df = df.append(details, ignore_index=True)
    X = df[list(df.columns)[3:]]
    X_test = np.asarray(X)
    y_pred = finalModel.predict(X_test)
    return(emotionMap[int(y_pred)])

In [3]:
fileName = "./sampleAudioFiles/angry.wav"

ipd.Audio(fileName)

output = findEmotion(fileName)

print("The Predicted Output is : ", output)

The Predicted Output is :  angry


In [4]:
fileName = "./sampleAudioFiles/sad.wav"

ipd.Audio(fileName)

output = findEmotion(fileName)

print("The Predicted Output is : ", output)

The Predicted Output is :  sad
